In [1]:
import pandas as pd

def generate_latex_table(csv_path, mixup, mosaic):
    import numpy as np
    df = pd.read_csv(csv_path)
    df_filtered = df[(df['mixup'] == mixup) & (df['mosaic'] == mosaic)]

    metrics_cols = ['Precision', 'Recall', 'F1 Score', 'mAP50', 'mAP50-95', 'Fitness']
    models = ['FCOS', 'RetinaNet', 'SSD', 'FasterRCNN', 'YOLO']
    rows = [
        ('real', 'real', 'Real / Real'),
        ('augmented', 'real', 'Aug. / Real'),
        ('real', 'augmented', 'Real / Aug.'),
        ('augmented', 'augmented', 'Aug. / Aug.'),
    ]

    lines = []
    lines.append('% table.tex')
    lines.append(r'\begin{table}[ht]')
    lines.append(r'\color{blue}')
    lines.append(r'  \centering')
    lines.append(r'  \begin{tabular}{l l ' + '*{6}{c}}')
    lines.append(r'    \toprule')
    lines.append(r'    \textbf{Model} & \textbf{Train/Test} & \multicolumn{6}{c}{\textbf{Metrics}} \\')
    lines.append('    \cmidrule(lr){3-8}')
    lines.append('    & & Precision & Recall & F1 Score & mAP50 & mAP50-95 & Fitness \\\\')
    lines.append('    \midrule')

    for idx, model in enumerate(models):
        lines.append(f'    \\multirow{{4}}{{*}}{{{model}}}')
        for jdx, (train, test, label) in enumerate(rows):
            subset = df_filtered[(df_filtered['model_name'] == model) &
                                 (df_filtered['train_dataset'] == train) &
                                 (df_filtered['test_dataset'] == test)]

            # Find the other row for the same test split to compare for bolding
            counterpart_train = 'augmented' if train == 'real' else 'real'
            counterpart = df_filtered[(df_filtered['model_name'] == model) &
                                      (df_filtered['train_dataset'] == counterpart_train) &
                                      (df_filtered['test_dataset'] == test)]

            values = []
            for col in metrics_cols:
                val = subset[col].values[0] if not subset.empty else np.nan
                cmp_val = counterpart[col].values[0] if not counterpart.empty else np.nan

                # Round values for comparison
                val_rounded = round(val, 2)
                cmp_rounded = round(cmp_val, 2)

                if np.isnan(val):
                    formatted = '--'
                elif np.isnan(cmp_val):
                    formatted = f'{val_rounded:.2f}'
                elif val_rounded > cmp_rounded:
                    formatted = f'\\textbf{{{val_rounded:.2f}}}'
                else:
                    formatted = f'{val_rounded:.2f}'
                values.append(formatted)

            line = f"      & {label:<17} & {' & '.join(values)} \\\\"
            lines.append(line)

            if jdx == 1:
                lines.append('    \\cmidrule(lr){2-8}')

        if idx < len(models) - 1:
            lines.append('    \\midrule')
        else:
            lines.append(r'    \bottomrule')

    lines.append('  \\end{tabular}')
    lines.append(f'  \\caption{{Evaluation metrics with on-the-fly data augmentation during training configured as mixup={mixup}, mosaic={mosaic}.}}')
    lines.append(f'  \\label{{tab:metrics-{mixup}-{mosaic}}}')
    lines.append('\\end{table}')

    return '\n'.join(lines)

if __name__ == '__main__':
    print(generate_latex_table('runs/metrics.csv', 0, 0),'\n')
    print(generate_latex_table('runs/metrics.csv', 0.5, 0),'\n')
    print(generate_latex_table('runs/metrics.csv', 0, 0.5),'\n')


% table.tex
\begin{table}[ht]
\color{blue}
  \centering
  \begin{tabular}{l l *{6}{c}}
    \toprule
    \textbf{Model} & \textbf{Train/Test} & \multicolumn{6}{c}{\textbf{Metrics}} \\
    \cmidrule(lr){3-8}
    & & Precision & Recall & F1 Score & mAP50 & mAP50-95 & Fitness \\
    \midrule
    \multirow{4}{*}{FCOS}
      & Real / Real       & 0.63 & 0.94 & 0.75 & 0.63 & 0.42 & 0.44 \\
      & Aug. / Real       & \textbf{0.65} & \textbf{0.97} & \textbf{0.78} & \textbf{0.65} & \textbf{0.44} & \textbf{0.46} \\
    \cmidrule(lr){2-8}
      & Real / Aug.       & 0.56 & 0.90 & 0.69 & 0.56 & 0.41 & 0.42 \\
      & Aug. / Aug.       & \textbf{0.69} & \textbf{0.98} & \textbf{0.81} & \textbf{0.69} & \textbf{0.53} & \textbf{0.55} \\
    \midrule
    \multirow{4}{*}{RetinaNet}
      & Real / Real       & 0.60 & 0.79 & 0.68 & 0.60 & 0.45 & 0.46 \\
      & Aug. / Real       & \textbf{0.61} & \textbf{0.81} & \textbf{0.70} & \textbf{0.61} & 0.45 & \textbf{0.47} \\
    \cmidrule(lr){2-8}
      & Real / A